In [4]:
import math
import random
import pandas as pd

class KDNode:
    def __init__(self, axis=0, left=None, right=None, bounds=None, is_leaf=False, points=None):
        self.axis = axis
        self.left = left
        self.right = right
        self.bounds = bounds
        self.is_leaf = is_leaf
        self.points = points if points is not None else pd.DataFrame()  # old_data
        self.new_count = 0  # new_data

class KDTree:
    def __init__(self, old_data: pd.DataFrame, delta=1, min_size=1e-2, alpha=0.05, window_size=50, k_bootstrap=100):
        
        if len(old_data) < window_size:
            raise ValueError("old_data is too short")
            
        elif len(old_data) > window_size:
            
            # new_data 슬라이싱
            old = old_data.iloc[:window_size].copy()
            new = old_data.iloc[window_size:].copy()

            raise ValueError("old_data is over the window_size", old, new)
            
            
        self.d = len(old_data.columns.tolist())
        self.delta = delta
        self.min_size = min_size
        self.alpha = alpha
        self.window_size = window_size
        self.k = k_bootstrap
        self.root = self.build_tree(old_data)

    def build_tree(self, points: pd.DataFrame, depth=0, bounds=None):
        

        axis = depth % self.d
        axis_name = points.columns[axis]  # 컬럼 이름 추출

        if bounds is None:
            min_bounds = [0.0] * self.k # 각 차원별 최소값
            max_bounds = [1.0] * self.k # 각 차원별 최대값

        else:
            min_bounds, max_bounds = bounds
            
        if points.empty:
            return KDNode(axis=axis, bounds=(min_bounds, max_bounds), is_leaf=True, points=None)

        if len(points) <= self.delta or (max_bounds[axis] - min_bounds[axis] <= self.min_size):
            return KDNode(axis=axis, bounds=(min_bounds, max_bounds), is_leaf=True, points=points)

        # 데이터 분할 (axis 컬럼 기준)
        midpoint = (min_bounds[axis] + max_bounds[axis]) / 2
        left_points = points[points[axis_name] < midpoint]
        right_points = points[points[axis_name] >= midpoint]

        left_max_bounds = max_bounds[:]
        left_max_bounds[axis] = midpoint
        right_min_bounds = min_bounds[:]
        right_min_bounds[axis] = midpoint

        return KDNode(
            axis=axis,
            bounds=(min_bounds, max_bounds),
            left=self.build_tree(left_points, depth + 1, (min_bounds, left_max_bounds)),
            right=self.build_tree(right_points, depth + 1, (right_min_bounds, max_bounds))
        )

    def _find_leaf(self, node, point):
        while not node.is_leaf:
            axis = node.axis
            midpoint = (node.bounds[0][axis] + node.bounds[1][axis]) / 2
            if point.iloc[axis] < midpoint:
                node = node.left
            else:
                node = node.right
        return node

    def _collect_leaf_nodes(self, node):
        if node is None:
            return []
        if node.is_leaf:
            return [node]
        return self._collect_leaf_nodes(node.left) + self._collect_leaf_nodes(node.right)

    def assign_new_data(self, new_data: pd.DataFrame):
        for leaf in self._collect_leaf_nodes(self.root):
            leaf.new_count = 0  # 초기화

        for _, point in new_data.iterrows():  # point: Series
            leaf = self._find_leaf(self.root, point)
            leaf.new_count += 1

    def _compute_kl_divergence(self):
        leaf_nodes = self._collect_leaf_nodes(self.root)
        num_boxes = len(leaf_nodes)
        total_old = sum(len(leaf.points) for leaf in leaf_nodes)
        total_new = sum(leaf.new_count for leaf in leaf_nodes)
        print(num_boxes)
        
        kl_sum = 0.0
        
            
        for leaf in leaf_nodes:
            p = (len(leaf.points) + 0.5) / (total_old + (num_boxes / 2)) if total_old > 0 else 0
            q = (leaf.new_count + 0.5) / (total_new + (num_boxes / 2)) if total_new > 0 else 0

            if p > 0 and q > 0:
                kl_sum += p * math.log(p / q)

        return kl_sum

    def _compute_bootstrap_kl(self, full_data: pd.DataFrame, seed=None):
        if len(full_data) % 2 != 0:
            raise ValueError("Full data must be of even length (2n).")
        
        n = len(full_data) // 2
        rng = random.Random(seed)
        kl_values = []
        
        
        for _ in range(self.k):
            
            # 복원추출로 2n개 샘플링
            sampled = full_data.sample(n=2 * n, replace=True, random_state=rng.randint(0, 1e6))

            # old/new 데이터 나누기 (DataFrame 슬라이싱)
            boot_old = sampled.iloc[:n]
            boot_new = sampled.iloc[n:]
            
            # 리프 초기화
            for leaf in self._collect_leaf_nodes(self.root):
                leaf.points = pd.DataFrame(columns=full_data.columns)
                leaf.new_count = 0
            
            # boot_old 배치
            for _, p in boot_old.iterrows():
                leaf = self._find_leaf(self.root, p)
                leaf.points = pd.concat([leaf.points, pd.DataFrame([p])], ignore_index=False)

            # boot_new 배치
            for _, p in boot_new.iterrows():
                leaf = self._find_leaf(self.root, p)
                leaf.new_count += 1

            kl = self._compute_kl_divergence()
            kl_values.append(kl)
            
        return kl_values
    
    def detect_drift(self, full_data: pd.DataFrame):
        d_hat = self._compute_kl_divergence()
        d_stars = self._compute_bootstrap_kl(full_data)
        print(d_hat)
        print('########################')
        print(d_stars)
        count_greater = sum(1 for d in d_stars if d >= d_hat) # d_hat보다 큰 d_star의 개수
        asl = count_greater / len(d_stars)  # ASL 계산
        
        if asl <= self.alpha:
            print("[Drift Detected] learner updates!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            return True
        
        else: 
            # 리프 초기화
            for leaf in self._collect_leaf_nodes(self.root):
                leaf.points = pd.DataFrame(columns=full_data.columns)
                   
            # old_data 재배치
            old = full_data.iloc[:self.window_size].copy()
            for _, p in old.iterrows():
                leaf = self._find_leaf(self.root, p)
                leaf.points = pd.concat([leaf.points, pd.DataFrame([p])], ignore_index=False)
                
            return False
        
        

    def print_tree(self, node=None, depth=0):
        if node is None:
            node = self.root
        indent = "  " * depth
        if node.is_leaf:
            print(f"{indent}Leaf | Bounds: {node.bounds} | Old: {len(node.points)} | New: {node.new_count}")
        else:
            print(f"{indent}Axis {node.axis}, Bounds: {node.bounds}")
            self.print_tree(node.left, depth + 1)
            self.print_tree(node.right, depth + 1)


In [ ]:
# 예제 데이터 준비
old = [[2, 3], [5, 4], [4, 7], [7, 2], [6, 5]]
new = [[2.1, 3.1], [5.5, 4.2], [4.5, 6.9], [6.8, 2.1], [6.2, 5.1]] 


# DataFrame으로 변환
old_data = pd.DataFrame(old, columns=['x', 'y'], index=[23342,23343,23344,23345,23346])
new_data = pd.DataFrame(new, columns=['x', 'y'], index=[23347,23348,23349,23350,23351])

# full_data도 DataFrame으로 합치기
full_data = pd.concat([old_data, new_data], ignore_index=False)

# KDTree 생성
tree = KDTree(old_data, delta=1, min_size=0.5, window_size=5)

# New data 할당
tree.assign_new_data(new_data)

# 원래 KL-divergence 계산
kl = tree._compute_kl_divergence()
print(f"Original KL-Divergence: {kl}")

# Bootstrap 기반 추정
bootstrap_kls = tree._compute_bootstrap_kl(full_data, seed=42)
print(f"Bootstrap KL Estimator (mean): {bootstrap_kls}")


### kdtree를 구성할 때는 제외하고 각 node instance에 points를 일일이 저장할 필요가 있을까? 
### 어차피 결국 계산하는것은 각 node에 들어있는 instance 개수인데?

In [5]:
from src.common.config      import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Lasso

##################################################################################################################################################################
# load and preprocess dataset
##################################################################################################################################################################

# load dataset
df_path = DATA_PATH[DATA_TYPE][DATA]
index   = DATASET[DATA_TYPE][DATA]['INDEX']
df      = pd.read_csv(f'../../../{df_path}', index_col = index)

# extract independent variable X 
input = DATASET[DATA_TYPE][DATA][f'INPUT_{VER}']
trgt  = DATASET[DATA_TYPE][DATA][f'TRGT']
X = df.loc[:, input]
y = df.loc[:, trgt]
y = y.values.ravel() # Note) y must be array!

# scaling dataset
scaler = MinMaxScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# set problem type
prob_type = DATASET[DATA_TYPE][DATA]['PROB_TYPE']

In [ ]:
init_tr = 37241
start_index = init_tr

old_data, new_data = pd.DataFrame(), pd.DataFrame()

# 슬라이딩 윈도우 실행 (X_new의 끝에서 24개씩 추가)
step_size = 12  # 24개씩 이동
old_data_ready = False
drift_indices = []
time = list(df.index)


while start_index + step_size <= len(X):
    
    # 새로운 batch 데이터 추가
    batch = X.iloc[start_index:start_index + step_size]
    
    
    
    ##### old_data 크기 조절
    if not old_data_ready:
        try:
            # old_data에 추가하기
            old_data = pd.concat([old_data, batch])

            start_index += step_size

            # kdqTree 형성
            tree = KDTree(old_data, delta=2, min_size=2**-3, alpha=0.01, window_size=20, k_bootstrap=100)
            old_data_ready = True

            continue

        except ValueError as e:

            if str(e) == "old_data is too short":
                print("❗ 데이터가 너무 작아서 KDTree 생성을 건너뜁니다.")
                continue

            elif e.args[0] == "old_data is over the window_size":
                print("❗ old_data가 window_size를 넘었습니다.")

                # window_size만큼 old_data에, 남은 데이터를 new_data에 할당
                old_data = e.args[1]
                new_data = e.args[2]

                tree = KDTree(old_data, delta=2, min_size=2**-3, alpha=0.01, window_size=20, k_bootstrap=100)
                old_data_ready = True

                continue

            else:
                raise  # 다른 예외는 그대로 올려보냄 
        
        
        
        
    ##### tree를 정상적으로 만들었으면 여기서 작업 수행
    
    
    
    
    ##### new_data 크기 조절
    
    # new_data에 추가하기
    new_data = pd.concat([new_data, batch])
   
    # new_data가 window보다 작을 경우
    if len(new_data) < tree.window_size:
        print('new data가 부족합니다~~~~~~~~~~~~~~~~~~~~')
        start_index += step_size
        continue
        
    # new_data가 window보다 클 경우
    if len(new_data) > tree.window_size:
        new_data = new_data.iloc[-tree.window_size:].copy()
   
    # New data 할당
    tree.assign_new_data(new_data)
    
    # drift 감지
    full_data = pd.concat([old_data, new_data])
    flag = tree.detect_drift(full_data)
    drift_idx = start_index + step_size
    
    if flag:
        old_data = new_data.copy()
        new_data = pd.DataFrame()
        tree.root = tree.build_tree(old_data)
        
        
        drift_indices.append(drift_idx)
    print(tree.root)    
    print(f'index {drift_idx} -> flag:', flag)
    # 다음 윈도우로 이동
    start_index += step_size

    
# time, y_pred, y_true를 데이터프레임으로 변환
pred_df = pd.DataFrame(time, columns=["time"])
pred_df['y_pred'] = y_pred
pred_df['y_true'] = y

# drift_flag 컬럼을 0으로 초기화
pred_df['drift_flag'] = 0
pred_df.loc[drift_indices, 'drift_flag'] = 1




# performance of test set
y_te_pred = pred_df['y_pred'].iloc[init_tr_end_idx:]
y_te_true = pred_df['y_true'].iloc[init_tr_end_idx:]

perf_te = {
        'mape':      np.round(mean_absolute_percentage_error(y_te_pred,y_te_true) * 100, 4),
        'mae':       np.round(mean_absolute_error(y_te_pred,y_te_true), 4),
        'rmse':      np.round(np.sqrt(mean_squared_error(y_te_pred,y_te_true)), 4),
        'r2':        np.round(r2_score(y_te_pred,y_te_true), 4),
        'ctq':       np.round(hit_ratio(y_te_pred,y_te_true, np.std(y_te_true)) * 100, 2)
    }

print(perf_te)
perf_df = pd.DataFrame(perf_te, index=[0])


# CSV 파일 저장

base_path = "C:/Users/lsjja/TM_MLOps"
res_df_pred_path  = os.path.join(base_path, RES_PATH['PRED_ROOT'] + RES_PATH['CDDA_SJL'])
res_df_perf_path  = os.path.join(base_path, RES_PATH['PERF_ROOT'] + RES_PATH['CDDA_SJL'])


res_df_pred_name = f'250402_APP_PDX_LASSO_kdqTree_REC_PRED_V1.csv'
res_df_perf_name = f'250402_APP_PDX_LASSO_kdqTree_REC_PERF_V1.csv'

pred_df.to_csv(res_df_pred_path + res_df_pred_name, index=False)
perf_df.to_csv(res_df_perf_path + res_df_perf_name, index=False)



❗ 데이터가 너무 작아서 KDTree 생성을 건너뜁니다.
❗ old_data가 window_size를 넘었습니다.
new data가 부족합니다~~~~~~~~~~~~~~~~~~~~
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
212
0.19308381484155582
########################
[0.10045791105179148, 0.019480917946931527, 0.060335506548952776, 0.05434659075023551, 0.055509426936103064, 0.09156416438335185, 0.04646209080945694, 0.059691030024155836, 0.05940213302258396, 0.10032706375601276, 0.06248282678591843, 0.08573880837813107, 0.13889727193846632, 0.08766760927447376, 0.1066733315418279, 0.04216630151830553, 0.10192786615667844, 0.04449878828379012, 0.13043418862615289, 0.04784169485425599, 0.08559161426439742, 0.0981

index 37337 -> flag: True
new data가 부족합니다~~~~~~~~~~~~~~~~~~~~
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
168
0.21711154076715786
########################
[0.09977077520255051, 0.11161005263781859, 0.06242383360387748, 0.08231902635354335, 0.06696679798873009, 0.08092981358163717, 0.16455663711889473, 0.06715220664112903, 0.07902032557850447, 0.0658573657909775, 0.01044254335431235, 0.04106015984287299, 0.04822137392683916, 0.07475387592842908, 0.07989153502776605, 0.13169388434230095, 0.08103106912172484, 0.12506385681440757, 0.11331815171162377, 0.031113689374667013, 0.09571819853200858, 0.09409353114797012, 0.13010052128795319, 0.063

82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
0.4226897786917685
########################
[0.2636688072290555, 0.04887507661020644, 0.1328330193443739, 0.10493545337164041, 0.15951178189935783, 0.052048230746191027, 0.19133034857374598, 0.13820668804491226, 0.11216660888309568, 0.11038793225277939, 0.12908005712037396, 0.07719523995427148, 0.24087190004845815, 0.032197224428991936, 0.10944281138658432, 0.12894365913944444, 0.11568624317888468, 0.10832701068897287, 0.13153578507051927, 0.05798839629109252, 0.3012397779965225, 0.195487527020399, 0.16685551044830949, 0.11489638020785083, 0.09915402462202802, 0.1309815180799232, 0.13687381139279053, 0.1113848168311314, 0.11701620761382743, 0.09261092275337773, 0.12467034447780936, 0.1012244204

148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
148
0.19913924334947958
########################
[0.07379193825799282, 0.07397439570998075, 0.1375872947526855, 0.09141082988931483, 0.04846512542481722, 0.104529954370443, 0.08806695076885121, 0.12167845259983685, 0.05988398280289856, 0.1040993865945393, 0.07607719299911594, 0.10112580592283335, 0.07873484002623214, 0.06497294994773707, 0.1185851008107784, 0.13830635435817407, 0.0959811078198741, 0.12440751514239662, 0.1911934390669433, 0.1435740086240358, 0.06710634097865809, 0.09376641432602366, 0.14449819052852345, 0.08096245311034286, 0.08295760899359071, 0.13441090649277965, 0.05130972349044509, 0.147367327193473, 0.09553157018954017, 0.1

233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
233
0.1856237271414582
########################
[0.04744942944418462, 0.03561157936693586, 0.07787195557505004, 0.0255701023239169, 0.06260645163566408, 0.04948549988466466, 0.049525809921349154, 0.09393163377586769, 0.06397977035377526, 0.06489440586344064, 0.06449690325428543, 0.08970983879389291, 0.08871462390955946, 0.12379218704040136, 0.07000772369690528, 0.06663978600026288, 0.04817527253755491, 0.06842698865969918, 0.121106105822125, 0.05698633981879253, 0.1209832305550445, 0.04013912372684854, 0.07590638039502154, 0.08133280331377425, 0.06731414472533893, 0.11132926086440792, 0.0407679322

...

251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
251
0.16024183845127069
########################
[0.033981712643604924, 0.03723944999401806, 0.018507555182388218, 0.07694363929217207, 0.07479417634845603, 0.05076701617094344, 0.021041033043440488, 0.048797085550998626, 0.020095819656059735, 0.05860885852454184, 0.08100231362159503, 0.043433934984725614, 0.14336564819113295, 0.039177553407655066, 0.08209751025603593, 0.06435578447113206, 0.08780104010839604, 0.03974873046616867, 0.04875790094264179, 0.04569884275631546, 0.13341830645802727, 0.0754578595040877, 0.09242608803446083, 0.10898292459212065, 0.03427075475130291, 0.04323410161597975, 0.04057559466250607, 0.08190106502219231, 0.036815395650718195, 0.09280996552105153, 0.08959191086645339, 0.052282797921433276, 0.031213583777285017, 0.072481

115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
115
0.38238232425615554
########################
[0.09866807608464714, 0.052585143293305985, 0.07047626592628542, 0.15645040978397598, 0.07871560473997247, 0.14545935736137522, 0.12344902374634806, 0.10065783184885277, 0.14798388385815564, 0.06974398633600731, 0.07856746939845345, 0.11607245094481998, 0.08594660416804423, 0.11603638469119316, 0.12084725429847094, 0.07476001650385447, 0.08312142769646987, 0.06520376982229736, 0.1323998589441241, 0.11117082308401875, 0.08347598829345422, 0.08077844598570505, 0.17634984428807896, 0.01934117924483663, 0.15020225429206321, 0.04320620371635327, 0.1265321270866201, 0.12071691378417354, 0.12538068813817005, 0.109627341611

239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
0.22936195298850748
########################
[0.052223163334435, 0.06720407655786297, 0.07326342405001439, 0.049910595739038326, 0.06885336695018662, 0.059080083608464735, 0.05138990324246046, 0.039427367350264274, 0.07880399781468798, 0.029601915130322752, 0.052261292708685264, 0.07548466455020504, 0.10736454778517888, 0.07373149503222537, 0.1147724064650938, 0.1048412461324155, 0.02580606344725197, 0.04544531298977972, 0.08790026239599487, 0.09751224819890149, 0.06643199513329122, 0.10112155984926974, 0.04368736383858505, 0.04621314930008575, 0.1066251469902117, 0.054677780652508785, 0.05647423435707578, 0.0587893307845359, 0.11190478152765909, 0.06697763763777728, 0.07446089101400907, 0.03322381116579666, 0.0548575979104475

239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
239
0.2597286993504515
########################
[0.0320685657796071, 0.03488856151559547, 0.058704919600346656, 0.06836577342319922, 0.04770376243759412, 0.045146353009119296, 0.03479884446788119, 0.05815579637430318, 0.048963065349004804, 0.03204905073349616, 0.1020403615316373, 0.051590429238444216, 0.06292266907831784, 0.04648032199585635, 0.03138130619580878, 0.03512197295274852, 0.04131732217945179, 0.08614824816477225, 0.05264814093330682, 0.06816959352463356, 0.04641450490409721, 0.05691007075697069, 0.034182760462421045, 0.04382485823940872, 0.04061373929710659, 0.09258644083420652, 0.020966975898581215, 0.0505295951704644, 0.0737024080565726, 0.043156275045639486

202
202
202
202
202
202
202
202
202
202
202
202
202
202
202
202
202
202
202
202
202
202
202
202
202
202
202
202
202
202
202
202
202
202
202
202
202
202
202
202
202
202
202
202
